In [ ]:
import pandas as pd
import os
import json
import glob
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="test_tk")

In [ ]:
os.makedirs('./results', exist_ok=True )

In [ ]:
data_path = '../download_with_python/data/'
cur_dir = os.getcwd()

In [ ]:
os.chdir(data_path)
files = glob.glob("*.json")
os.chdir(cur_dir)

In [ ]:
crimes_dict = {}
offenders_dict = {}
env_dict = {}
i = 0

for file in files:
    name = file.split(sep="_")[0]
    if name == 'crime':
        crimes_dict[name+f'_{i}'] = pd.read_json((data_path+file), lines=True)
    elif name == 'offenders':
        offenders_dict[name+f'_{i}'] = pd.read_json((data_path+file), lines=True)
        # drop registered as homeless offenders
        offenders_dict[name+f'_{i}'] = offenders_dict[name+f'_{i}'][~(offenders_dict[name+f'_{i}']['block'] == '(registered as homeless)')]
        # replace anonymized house number with 0 (replaces X with 0)
        offenders_dict[name+f'_{i}']['block'] = offenders_dict[name+f'_{i}']['block'].str.replace('X','0') + ' CHICAGO'
    elif name == 'envcomplaint':
        env_dict[name+f'_{i}'] = pd.read_json((data_path+file), lines=True)
    else:
        print(f'error on {file}')


In [ ]:
# TO DO
# make merger function to merge like for like

In [ ]:
# define geojsons
with open('../../geojsons/Boundaries - Community Areas (current).geojson') as f:
    commarea = json.load(f)

In [ ]:
def merge_df(dictionary):
    merged_df = pd.DataFrame()
    for datfram in dictionary:
        merged_df= pd.concat([merged_df,dictionary[datfram]],axis=0)
    return merged_df

In [ ]:
merged_crimes = merge_df(crimes_dict)
merged_offenders = merge_df(offenders_dict)
merged_envcomplaints = merge_df(env_dict)

In [ ]:
def get_commarea_env(dataframe):
    dataframe['neighborhood']=''
    for row in dataframe.index:
        point = Point(dataframe['location.coordinates'][row][0],dataframe['location.coordinates'][row][1])
        for feature in commarea['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                dataframe['neighborhood'][row] = feature['properties']['community']

In [ ]:
def getlatlong(dataframe):
    for row in dataframe.index:
        try:
            location = geolocator.geocode(dataframe['block'][row])
            latlong = [location.latitude, location.longitude]
            point = Point(latlong[1], latlong[0])
        except:
            dataframe['neighborhood'][row] = 'no match to osm'

In [ ]:
def get_commarea_offenders(dataframe):
    dataframe['neighborhood']=''
    i=0
    for row in dataframe.index:
        try:
            location = geolocator.geocode(dataframe['block'][row])
            latlong = [location.latitude, location.longitude]
            point = Point(latlong[1], latlong[0])
            for feature in commarea['features']:
                polygon = shape(feature['geometry'])
                if polygon.contains(point):
                    dataframe['neighborhood'][row] = feature['properties']['area_numbe']
                else:
                    dataframe['neighborhood'][row] = 'no match to neighborhood'
        except:
            dataframe['neighborhood'][row] = 'no match to osm'
        print(f'{round((i/dataframe.shape[0]),4)*100}% done')
        i+=1


In [ ]:
location = geolocator.geocode(merged_offenders['block'][10])
latlong = [location.latitude, location.longitude]
point = Point(latlong[1], latlong[0])
for feature in commarea['features']:
    polygon = shape(feature['geometry'])
if polygon.contains(point):
    merged_offenders['neighborhood'][5] = feature['properties']['area_numbe']
else:
    merged_offenders['neighborhood'][5] = 'no match to neighborhood'

In [ ]:
get_commarea_env(merged_envcomplaints)
get_commarea_offenders(merged_offenders)

In [ ]:
merged_crimes.to_csv('./results/crimes_cleaned.csv', index=False)
merged_offenders.to_csv('./results/offeders_cleaned.csv', index=False)
merged_envcomplaints.to_csv('./results/crimes_envcomplaints.csv', index=False)

In [ ]:
grouped_offenders_by_commarea = merged_offenders.groupby('neighborhood').agg(total_offenders_in_ca=('last','count')).reset_index()
grouped_offenders_by_commarea.to_csv('./results/offenders_by_neighborhood.csv', index=False)

In [ ]:
grouped_complaints_by_commarea = merged_envcomplaints.groupby('neighborhood').agg(total_complaints_in_ca=('complaint_id','count')).reset_index()
grouped_complaints_by_commarea.to_csv('./results/envcomplaints_by_neighborhood.csv', index=False)

In [ ]:
grouped_offenders_by_commarea.head()

In [ ]:
merged_crimes.dtypes

In [ ]:
big_df = merged_crimes.merge(grouped_offenders_by_commarea, how='left', on='neighborhood')